## Exploring the GPT Chat Completion Model

In this notebook, I delve into the workings of a standard GPT chat completion model, evaluating both its advantages and disadvantages.

## Advantages:

### 1. Simple and Understandable Code:
   The implementation of the GPT chat completion model is characterized by its simplicity and ease of understanding. The code is designed to be accessible, making it suitable for users at various levels of familiarity with natural language processing.

### 2. Efficient with Small Prompts:
   Despite its concise prompts, the model exhibits remarkable efficiency. It consistently provides accurate and contextually relevant completions, demonstrating its ability to understand and respond effectively to limited input.

### 3. High Accuracy:
   Notably, the GPT chat completion model boasts high accuracy. The generated responses are contextually coherent, showcasing the model's proficiency in capturing the nuances of language and producing meaningful outputs.

## Disadvantages:

### 1. Costly:
   One significant drawback is the associated cost of utilizing this GPT model. Implementing this model may incur substantial expenses, especially for projects with extensive usage. Users should be mindful of the financial implications when opting for this solution.

### 2. Token Limitation:
   Another limitation to consider is the constraint on the number of tokens that can be processed in a single request. If the input surpasses this token limit, it may result in errors, necessitating careful management of prompt length to avoid unexpected disruptions.

In summary, the GPT chat completion model exhibits simplicity, accuracy, and efficiency with concise prompts. However, users should carefully consider the cost implications and token limitations to determine the model's suitability for specific use cases.


### Instaling the required libraries

In [49]:
# !pip install tiktoken openai pinecone-client langchain
# !pip install pypdf


### Importing the required libraries

In [171]:
from langchain.llms import OpenAI
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
)
import openai
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
import os
openai.api_key =input('Enter your gpt api key here:--->')

### Code Starts Here

In [270]:
# Checking the file format and reading it.
def process_docs(file_path):
    documents = []
    if file_path.lower().endswith('.pdf'):
        loader = PyPDFLoader(file_path)
        documents.extend(loader.load())
    elif file_path.lower().endswith('.docx') or file_path.lower().endswith('.doc'):
        loader = Docx2txtLoader(file_path)
        documents.extend(loader.load())
    return documents

In [271]:
def easy_way_gpt(list_page):
    completion = openai.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
      {"role": "system", "content": """Given the following resume, please parse it and provide the extracted information in a **Structured JSON format**. Ensure the JSON includes details such as personal information, education history, skills, summary, and work history.
      Example:Example:
    {
        "address": {
            "city": "Specify City",
            "country": "Specify Country Code",
            "state": "Specify State Abbreviation"
        },
        "education_history": [
            {
                "degree": "Specify Degree",
                "from_date": "Specify Start Date",
                "name": "Specify School/Institute Name",
                "to_date": "Specify End Date"
            }
        ],
        "email": "Specify Email",
        "first_name": "Specify First Name",
        "last_name": "Specify Last Name",
        "phone": "Specify Phone Number",
        "skills": [
            {"skill": "Specify Skill 1"},
             ...]}
             "work_history": [
        {
            "company": "Company name 1",
            "description":"All work done"
            "from_date": "Start Date",
            "title": "Job Title",
            "to_date": "End "}"""},
      {"role": "user", "content":'\n'.join(list_page)}
        ]
      )
    conversation=dict(completion.choices[0].message)['content']
    return coversation
    

In [269]:
DIR_PATH = "/home/shivi/Desktop/resume-parser"
for file_name in os.listdir(DIR_PATH):
    file_path = os.path.join(DIR_PATH, file_name)
    processed_documents = process_docs(file_path)
    list_page=[]
    for i in processed_documents:
        list_page.append(i.page_content)
    convo=conversation.splitlines()
    resume_data_str = '\n'.join(convo)
    resume_data_str=resume_data_str.replace('```json\n','').replace('\n```','')
    resume_data = json.loads(resume_data_str)
    modified_skills = [{"skill": skill.strip('\"')} for skill in resume_data["skills"]]
    resume_data["skills"] = modified_skills
    file_path_save_json= os.path.join("/home/shivi/Desktop/parsed_resume", file_name.split('.')[0]+".json")
    with open(file_path_save_json, "w") as json_file:
        json.dump(resume_data, json_file, indent=4)
    
        